# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 7 2022 9:30AM,248145,10,8622337,"Citieffe, Inc.",Released
1,Oct 7 2022 9:27AM,248143,19,8608263,Eywa Pharma Inc.,Released
2,Oct 7 2022 9:27AM,248143,19,8608264,Eywa Pharma Inc.,Released
3,Oct 7 2022 9:26AM,248144,12,HH-37360,Hush Hush,Released
4,Oct 7 2022 9:26AM,248144,12,HH-37361,Hush Hush,Released
5,Oct 7 2022 9:26AM,248144,12,HH-37362,Hush Hush,Released
6,Oct 7 2022 9:26AM,248144,12,HH-37363,Hush Hush,Released
7,Oct 7 2022 9:26AM,248144,12,HH-37364,Hush Hush,Released
8,Oct 7 2022 9:26AM,248144,12,HH-37365,Hush Hush,Released
9,Oct 7 2022 9:26AM,248144,12,HH-37366,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,248140,Released,59
26,248141,Released,8
27,248143,Released,2
28,248144,Released,14
29,248145,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248140,NaN,NaN,59.0
248141,NaN,NaN,8.0
248143,NaN,NaN,2.0
248144,NaN,NaN,14.0
248145,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248037,7.0,0.0,1.0
248071,0.0,1.0,0.0
248074,0.0,0.0,42.0
248085,0.0,0.0,1.0
248097,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248037,7,0,1
248071,0,1,0
248074,0,0,42
248085,0,0,1
248097,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248037,7,0,1
1,248071,0,1,0
2,248074,0,0,42
3,248085,0,0,1
4,248097,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248037,7,,1
1,248071,,1,
2,248074,,,42
3,248085,,,1
4,248097,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc."
1,Oct 7 2022 9:27AM,248143,19,Eywa Pharma Inc.
3,Oct 7 2022 9:26AM,248144,12,Hush Hush
17,Oct 7 2022 9:25AM,248141,10,Bio-PRF
25,Oct 7 2022 9:13AM,248140,10,ISDIN Corporation
84,Oct 7 2022 9:02AM,248138,10,ISDIN Corporation
111,Oct 7 2022 9:01AM,248139,16,Sartorius Lab Products and Service
113,Oct 7 2022 8:58AM,248137,19,"GUSA Granules USA, Inc."
114,Oct 7 2022 8:50AM,248135,10,ISDIN Corporation
135,Oct 7 2022 8:40AM,248134,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",,,1
1,Oct 7 2022 9:27AM,248143,19,Eywa Pharma Inc.,,,2
2,Oct 7 2022 9:26AM,248144,12,Hush Hush,,,14
3,Oct 7 2022 9:25AM,248141,10,Bio-PRF,,,8
4,Oct 7 2022 9:13AM,248140,10,ISDIN Corporation,,,59
5,Oct 7 2022 9:02AM,248138,10,ISDIN Corporation,,,27
6,Oct 7 2022 9:01AM,248139,16,Sartorius Lab Products and Service,,,2
7,Oct 7 2022 8:58AM,248137,19,"GUSA Granules USA, Inc.",,,1
8,Oct 7 2022 8:50AM,248135,10,ISDIN Corporation,,1,20
9,Oct 7 2022 8:40AM,248134,19,"Innogenix, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",1,,
1,Oct 7 2022 9:27AM,248143,19,Eywa Pharma Inc.,2,,
2,Oct 7 2022 9:26AM,248144,12,Hush Hush,14,,
3,Oct 7 2022 9:25AM,248141,10,Bio-PRF,8,,
4,Oct 7 2022 9:13AM,248140,10,ISDIN Corporation,59,,
5,Oct 7 2022 9:02AM,248138,10,ISDIN Corporation,27,,
6,Oct 7 2022 9:01AM,248139,16,Sartorius Lab Products and Service,2,,
7,Oct 7 2022 8:58AM,248137,19,"GUSA Granules USA, Inc.",1,,
8,Oct 7 2022 8:50AM,248135,10,ISDIN Corporation,20,1,
9,Oct 7 2022 8:40AM,248134,19,"Innogenix, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",1,,
1,Oct 7 2022 9:27AM,248143,19,Eywa Pharma Inc.,2,,
2,Oct 7 2022 9:26AM,248144,12,Hush Hush,14,,
3,Oct 7 2022 9:25AM,248141,10,Bio-PRF,8,,
4,Oct 7 2022 9:13AM,248140,10,ISDIN Corporation,59,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Oct 7 2022 9:27AM,248143,19,Eywa Pharma Inc.,2.0,NaN,NaN
2,Oct 7 2022 9:26AM,248144,12,Hush Hush,14.0,NaN,NaN
3,Oct 7 2022 9:25AM,248141,10,Bio-PRF,8.0,NaN,NaN
4,Oct 7 2022 9:13AM,248140,10,ISDIN Corporation,59.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Oct 7 2022 9:27AM,248143,19,Eywa Pharma Inc.,2.0,0.0,0.0
2,Oct 7 2022 9:26AM,248144,12,Hush Hush,14.0,0.0,0.0
3,Oct 7 2022 9:25AM,248141,10,Bio-PRF,8.0,0.0,0.0
4,Oct 7 2022 9:13AM,248140,10,ISDIN Corporation,59.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1488817,116.0,1.0,0.0
12,248144,14.0,0.0,0.0
15,496111,43.0,0.0,7.0
16,2977444,46.0,2.0,0.0
19,744414,5.0,0.0,0.0
20,496156,1.0,1.0,0.0
21,496253,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1488817,116.0,1.0,0.0
1,12,248144,14.0,0.0,0.0
2,15,496111,43.0,0.0,7.0
3,16,2977444,46.0,2.0,0.0
4,19,744414,5.0,0.0,0.0
5,20,496156,1.0,1.0,0.0
6,21,496253,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,116.0,1.0,0.0
1,12,14.0,0.0,0.0
2,15,43.0,0.0,7.0
3,16,46.0,2.0,0.0
4,19,5.0,0.0,0.0
5,20,1.0,1.0,0.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,12,Released,14.0
2,15,Released,43.0
3,16,Released,46.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,7.0,0.0,0.0,0.0,0.0
Executing,1.0,0.0,0.0,2.0,0.0,1.0,0.0
Released,116.0,14.0,43.0,46.0,5.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,2.0,0.0,1.0,0.0
2,Released,116.0,14.0,43.0,46.0,5.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,2.0,0.0,1.0,0.0
2,Released,116.0,14.0,43.0,46.0,5.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()